# Part 2: Streaming application using Spark Structured Streaming  
In this task, you will implement Spark Structured Streaming to consume the data from task 1 and perform a prediction.    
Important:   
-	This task uses PySpark Structured Streaming with PySpark Dataframe APIs and PySpark ML.  
-	You also need your pipeline model from A2A to make predictions and persist the results.  

1.	Write code to create a SparkSession, which 1) uses four cores with a proper application name; 2) use the Melbourne timezone; 3) ensure a checkpoint location has been set.


In [9]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'

# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[4]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Assignment2B"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name) \
                        .set("spark.sql.streaming.checkpointLocation", "checkpoints")

# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).config("spark.sql.session.timeZone", "GMT+10").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

from pyspark.sql import functions as F

2.	Write code to define the data schema for the data files, following the data types suggested in the metadata file. Load the static datasets (e.g. building information) into data frames. (You can reuse your code from 2A.)


In [10]:
# Adapted from GPT
from pyspark.sql.types import (
    StructType, StructField,
    IntegerType, StringType, DecimalType, TimestampType
)

# 1. Meters Table
meters_schema = StructType([
    StructField("building_id", IntegerType(), False),
    StructField("meter_type", StringType(), False),   # Char(1) -> StringType
    StructField("ts", TimestampType(), False),
    StructField("value", DecimalType(15, 4), False),
    StructField("row_id", IntegerType(), False)
])

# 2. Buildings Table
buildings_schema = StructType([
    StructField("site_id", IntegerType(), False),
    StructField("building_id", IntegerType(), False),
    StructField("primary_use", StringType(), True),
    StructField("square_feet", IntegerType(), True),
    StructField("floor_count", IntegerType(), True),
    StructField("row_id", IntegerType(), False),
    StructField("year_built", IntegerType(), True),
    StructField("latent_y", DecimalType(6, 4), True),
    StructField("latent_s", DecimalType(6, 4), True),
    StructField("latent_r", DecimalType(6, 4), True)
])

# 3. Weather Table
# weather_schema = StructType([
#     StructField("site_id", StringType(), False),
#     StructField("timestamp", StringType(), False),
#     StructField("air_temperature", StringType(), True),
#     StructField("cloud_coverage", StringType(), True), # Is an Integer, but ends with a ".0", so read as a DecimalType
#     StructField("dew_temperature", StringType(), True),
#     StructField("sea_level_pressure", StringType(), True),
#     StructField("wind_direction", StringType(), True), # Is an Integer, but ends with a ".0", so read as a DecimalType
#     StructField("wind_speed", StringType(), True),
#     StructField("weather_ts", StringType(), False) # new field
# ])

weather_schema = StructType([
    StructField("site_id", StringType(), False),
    StructField("timestamp", TimestampType(), False),
    StructField("air_temperature", DecimalType(5, 3), True),
    StructField("cloud_coverage", DecimalType(5, 3), True), # Is an Integer, but ends with a ".0", so read as a DecimalType
    StructField("dew_temperature", DecimalType(5, 3), True),
    StructField("sea_level_pressure", DecimalType(8, 3), True),
    StructField("wind_direction", DecimalType(5, 3), True), # Is an Integer, but ends with a ".0", so read as a DecimalType
    StructField("wind_speed", DecimalType(5, 3), True),
    StructField("weather_ts", TimestampType(), False) # new field
])


# buildings_df = spark.read.csv(
#     "data/building_information.csv",
#     header=True,
#     schema=buildings_schema
# )

buildings_df = spark.read.csv(
    "data/new_building_information.csv",
    header=True,
    schema=buildings_schema
)

weather_df = spark.read.csv(
    "data/weather.csv",
    header=True,
    schema=weather_schema
)


3.	Using the Kafka topic from the producer in Task 1, ingest the streaming data into Spark Streaming, assuming all data comes in the String format. Except for the 'weather_ts' column, you shall receive it as an Int type. Load the new building information CSV file into a dataframe. Then, the data frames should be transformed into the proper formats following the metadata file schema, similar to assignment 2A.


In [11]:


#configuration
hostip = "192.168.0.6"#"10.192.90.63" #change to your machine IP address
topic = 'weather_data'

df_raw = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", f'{hostip}:9092') \
    .option("subscribe", topic) \
    .load()

df_str = df_raw.selectExpr("CAST(value AS STRING) as json_str")
# df_parsed = df_str.withColumn(
#     "data",
#     F.from_json(F.col("json_str"), F.ArrayType(weather_schema))
# )
# df_exploded = df_parsed.select(F.explode(F.col("data")).alias("record"))
# df_final = df_exploded






weather_stream = (
    df_str
    .withColumn("data", F.from_json(F.col("json_str"), F.ArrayType(weather_schema)))
    .select(F.explode(F.col("data")).alias("r"))
    .select("r.*")
)


4.	Use a watermark on weather_ts, if data points are received 5 seconds late, discard the data.

In [12]:
weather_stream = weather_stream.withWatermark("weather_ts", '5 seconds')

5.	Perform the necessary transformation you used in A2A. (note: every student may have used different features, feel free to reuse the code you have written in A2A. If you built an end-to-end pipeline, you can ignore this task.) 

In [13]:

# from A2A which was from GPT
# Get global_means, site_month_means, site_means
# weather_df is history, weather_stream is current

# Split timestamp to date, month, time bucket
weather_df = weather_df.withColumn("date", F.to_date("timestamp")).withColumn(
    "time",
    F.when(F.hour("timestamp") <= 5, "0-6h")
     .when(F.hour("timestamp") <= 11, "6-12h")
     .when(F.hour("timestamp") <= 17, "12-18h")
     .when(F.hour("timestamp") <= 23, "18-24h")
).withColumn("month", F.month("timestamp"))

# Choose which columns to impute
impute_cols = [
    "air_temperature",
    "cloud_coverage",
    "dew_temperature",
    "sea_level_pressure",
    "wind_direction",
    "wind_speed"
]

# Compute global_means, site_month_means, site_means
global_means = weather_df.select(
    *[F.mean(c).alias(c) for c in impute_cols]
).first().asDict()

site_month_means = weather_df.groupBy("site_id", "month").agg(
    *[F.mean(c).alias(f"{c}_site_month_mean") for c in impute_cols]
)

site_means = weather_df.groupBy("site_id").agg(
    *[F.mean(c).alias(f"{c}_site_mean") for c in impute_cols]
)
    
# Skip Garbage collection
# del site_month_means
# del site_means
# del global_means
# spark.catalog.clearCache()

# Transform weather_stream
# Split timestamp to date, month, time bucket
weather_stream = weather_stream.withColumn("date", F.to_date("timestamp")).withColumn(
    "time",
    F.when(F.hour("timestamp") <= 5, "0-6h")
     .when(F.hour("timestamp") <= 11, "6-12h")
     .when(F.hour("timestamp") <= 17, "12-18h")
     .when(F.hour("timestamp") <= 23, "18-24h")
).withColumn("month", F.month("timestamp"))


# Step 1: site_id + month
weather_stream = weather_stream.join(site_month_means, on=["site_id", "month"], how="left")
for c in impute_cols:
    weather_stream = weather_stream.withColumn(
        c, F.coalesce(c, F.col(f"{c}_site_month_mean"))
    ).drop(f"{c}_site_month_mean")
    
# Step 2: site_id
weather_stream = weather_stream.join(site_means, on="site_id", how="left")
for c in impute_cols:
    weather_stream = weather_stream.withColumn(
        c, F.coalesce(c, F.col(f"{c}_site_mean"))
    ).drop(f"{c}_site_mean")

# Step 3: global fallback
for c in impute_cols:
    weather_stream = weather_stream.withColumn(
        c, F.coalesce(c, F.lit(global_means[c]))
    )
# Aggregate by time bucket
weather_stream = (
    weather_stream
#     .withWatermark("weather_ts", "5 seconds")
    .groupBy(
        "site_id", "date", "time", "month",
        F.window("weather_ts", "5 seconds")
    )
    .agg(
        F.mean("air_temperature").cast(DecimalType(5, 3)).alias("air_temperature"),
        F.mean("cloud_coverage").cast(DecimalType(5, 3)).alias("cloud_coverage"),
        F.mean("dew_temperature").cast(DecimalType(5, 3)).alias("dew_temperature"),
        F.mean("sea_level_pressure").cast(DecimalType(8, 3)).alias("sea_level_pressure"),
        F.mean("wind_direction").cast(DecimalType(5, 3)).alias("wind_direction"),
        F.mean("wind_speed").cast(DecimalType(5, 3)).alias("wind_speed"),     
    )
)

# Add custom columns
weather_stream = (
    weather_stream
    .withColumn("dew_depression", F.col("air_temperature") - F.col("dew_temperature"))
    .withColumn("nonideal_temp", (F.col("air_temperature") - 18)**2)
    .drop("air_temperature")
    .drop("dew_temperature")
)

# No need to add median temp and peak-offpeak as our pipeline model later does not use them
feature_df = buildings_df.join(weather_stream, ["site_id"])


6.	Load your pipeline model and perform the following aggregations:  
a)	Print the prediction from your model as a stream comes in.  
b)	Every 7 seconds, print the total energy consumption for each 6-hour interval, aggregated by building, and print 20 records. (Note: This is simulating energy data each day in a week)  
c)	Every 14 seconds, for each site, print the daily total energy consumption.  

In [14]:
# i want to predict (and print) from my earlier created model as the stream comes in. the model was earlier saved as:
# ```
# # Define the path to save the model
# model_path = "models/best_model_rmsle"

# # Save the better trained model. 
# if rf_cv_metrics["rmsle"] < gbt_cv_metrics["rmsle"]:
#     rf_cv_model.bestModel.save(model_path)
# else:
#     gbt_cv_model.bestModel.save(model_path)
# ```
# where exactly do i load and run the model (was it model.transform(feature_df)?) to predict and append another column "value"?
# feature_df is df_final as above, with some imputation and a join done. 


In [15]:
# 6a
# --- 1. Spark + model setup ---
# from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

model = PipelineModel.load("models/best_model_rmsle")

# --- 3. Apply model ---
predictions = model.transform(feature_df).withColumnRenamed("prediction", "log_power_usage")


In [17]:
checkpoint_dir = os.path.abspath("checkpoints/weather_stream")
os.makedirs(checkpoint_dir, exist_ok=True)

def foreach_batch_function(df, epoch_id):
    df.show(10,False)
# --- 4. Output ---
query = (
    predictions
        .select("features", "log_power_usage")
        .writeStream
#         .foreachBatch(foreach_batch_function)
        .format("console")
        .outputMode("append")
        .option("checkpointLocation", checkpoint_dir)
        .option("truncate", False)
        .start()
)

query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# def foreach_batch_function(df, epoch_id):
#     df.show(10,False)
# query = (
#     weather_stream.writeStream
#         .format("console")
#         .foreachBatch(foreach_batch_function)
#         .outputMode("append")
#         .start()
# )

# query.awaitTermination()

In [ ]:
# 6b


In [ ]:
# 6c


7.	Save the data from 6 to Parquet files as streams. (Hint: Parquet files support streaming writing/reading. The file keeps updating while new batches arrive.)

In [ ]:
# 7a(save 6a)


In [ ]:
# 7b(save 6b)


In [ ]:
# 7c(save 6c)

8.	Read the parquet files from task 7 as data streams and send them to Kafka topics with appropriate names.
(Note: You shall read the parquet files as a streaming data frame and send messages to the Kafka topic when new data appears in the parquet file.)

In [ ]:
# Stream 1


In [ ]:
# Stream 2


In [ ]:
# Stream 3
